In [1]:
import kagglehub
irkaal_foodcom_recipes_and_reviews_path = kagglehub.dataset_download('irkaal/foodcom-recipes-and-reviews')
print('Data source import complete.')

Data source import complete.


In [2]:
import pandas as pd
import numpy as np
import ast

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

**Análisis de Datos**

In [4]:
og_recipes = pd.read_csv('recipes.csv')
og_recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [5]:
recipes = og_recipes.drop(['AuthorName','Description','CookTime', 'PrepTime', 'TotalTime', 'DatePublished',
                           'Images','RecipeIngredientQuantities', 'ReviewCount',
                           "RecipeYield", 'RecipeInstructions'], axis=1)
recipes.fillna(0, inplace=True)

**Flags restrictivas**

In [6]:
def is_x_flag(ingredients, keywords):
    if isinstance(ingredients, str):
        ingredients_str = ingredients.lower()
        for keyword in keywords:
            if keyword.lower() in ingredients_str:
                return 0
    return 1

Vegetariano

In [7]:
animal_keywords = [
    "beef", "pork", "chicken", "turkey", "bacon", " ham", "lamb", "veal", "duck", "goose",
    "sausage", "prosciutto", "salami", "steak", "ribs", "wings", "drumstick", "fillet", "cutlet",
    "tenderloin", "liver", "giblets", "gizzard", "tripe", "mutton", "venison", "rabbit",
    "kangaroo", "bison", "quail", "boar", "snail", "frog", "foie gras", "chorizo", "anchovy",
    "anchovies", "tuna", "salmon", "sardine", "trout", "mackerel", "fish", "seafood", "crab",
    "lobster", "shrimp", "prawn", "octopus", "squid", "clam", "oyster", "scallop", "eel",
    "halibut", "snapper", "herring", "catfish", "tilapia", "muscles"
]

In [8]:
recipes['is_vegetarian'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(animal_keywords,))
print(recipes[['RecipeIngredientParts', 'is_vegetarian']].head())

                               RecipeIngredientParts  is_vegetarian
0  c("blueberries", "granulated sugar", "vanilla ...              1
1  c("saffron", "milk", "hot green chili peppers"...              0
2  c("sugar", "lemons, rind of", "lemon, zest of"...              1
3  c("extra firm tofu", "eggplant", "zucchini", "...              1
4  c("plain tomato juice", "cabbage", "onion", "c...              1


Libre de lactosa

In [9]:
dairy_keywords = [
    "whole milk", "skim milk", "low-fat milk", "condensed milk", "evaporated milk",
    "powdered milk", "buttermilk", "heavy cream", "whipping cream", "sour cream",
    "half and half", "cheese", "cheddar", "mozzarella", "parmesan", "brie", "feta",
    "gouda", "provolone", "ricotta", "yogurt", "greek yogurt", "kefir", "curd",
    "paneer", "butter", "clarified butter", "ghee", "whey", "casein", "lactose"
]

In [10]:
recipes['is_lactose_free'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(dairy_keywords,))
print(recipes[['RecipeIngredientParts', 'is_lactose_free']].head())

                               RecipeIngredientParts  is_lactose_free
0  c("blueberries", "granulated sugar", "vanilla ...                0
1  c("saffron", "milk", "hot green chili peppers"...                0
2  c("sugar", "lemons, rind of", "lemon, zest of"...                1
3  c("extra firm tofu", "eggplant", "zucchini", "...                1
4  c("plain tomato juice", "cabbage", "onion", "c...                1


Vegano

In [11]:
non_vegan_keywords = [
    "egg", "albumen", "honey", "bee pollen", "jelly", "propolis", "gelatin",
    "lard", "tallow", "suet", "rennet", "cochineal", "carmine", "shellac",
    "broth", "stock", "isinglass", "glycerin","castoreum"
] + animal_keywords + dairy_keywords

In [12]:
recipes['is_vegan'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(non_vegan_keywords,))
print(recipes[['RecipeIngredientParts', 'is_vegan']].head())

                               RecipeIngredientParts  is_vegan
0  c("blueberries", "granulated sugar", "vanilla ...         0
1  c("saffron", "milk", "hot green chili peppers"...         0
2  c("sugar", "lemons, rind of", "lemon, zest of"...         1
3  c("extra firm tofu", "eggplant", "zucchini", "...         0
4  c("plain tomato juice", "cabbage", "onion", "c...         1


Libre de gluten

In [13]:
gluten_keywords = [
    "wheat", "white flour", "all-purpose flour", "enriched flour",
    "bread flour", "cake flour", "self-rising flour", "semolina", "spelt",
    "durum", "farro", "couscous", "bulgur", "kamut", "graham flour", "triticale",
    "barley", "malt", "rye", "pumpernickel", "rye flour", "seitan", "gluten",
    "soy sauce", "teriyaki sauce", "hoisin sauce", "beer", "ale", "lager", "stout",
    "bread", "buns", "bagels", "croissant", "crackers", "cookies", "cakes", "pastries",
    "biscuits", "noodles", "pasta", "macaroni", "lasagna", "ravioli", "dumplings",
    "cereal", "granola", "bran"
]

In [14]:
recipes['is_gluten_free'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(gluten_keywords,))
print(recipes[['RecipeIngredientParts', 'is_gluten_free']].head())

                               RecipeIngredientParts  is_gluten_free
0  c("blueberries", "granulated sugar", "vanilla ...               1
1  c("saffron", "milk", "hot green chili peppers"...               1
2  c("sugar", "lemons, rind of", "lemon, zest of"...               1
3  c("extra firm tofu", "eggplant", "zucchini", "...               0
4  c("plain tomato juice", "cabbage", "onion", "c...               1


Otros

In [15]:
nut_keywords = ["nut", "almond", "cashew", "pecan", "pistachio", "macadamia"]
recipes['is_nut_free'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(nut_keywords,))

soy_keywords = ["soy","edamame", "tofu", "tempeh"]
recipes['is_soy_free'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(soy_keywords,))

shellfish_keywords = ["shrimp", "prawn", "crab", "lobster", "scallop", "clam", "mussel", "oyster"]
recipes['is_shellfish_free'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(shellfish_keywords,))

halal_keywords = ["pork", "bacon", "ham", "gelatin", "lard", "alcohol", "wine", "beer", "rum", "brandy"]
recipes['is_halal'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(halal_keywords,))

kosher_keywords = ["pork", "shellfish", "catfish", "rabbit", "gelatin"]
recipes['is_kosher'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(kosher_keywords,))

alcohol_keywords = ["alcohol", "ethanol", "wine", "beer", "vodka", "rum", "whiskey", "brandy", "liqueur"]
recipes['is_alcohol_free'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(alcohol_keywords,))

high_fodmap_keywords = [
    "garlic", "onion", "wheat", "rye", "legumes", "lentils", "chickpeas", "beans", "apples", "pears",
    "honey", "milk", "yogurt", "cheese", "sorbitol", "mannitol", "xylitol"
]
recipes['is_high_fodmap'] = recipes['RecipeIngredientParts'].apply(is_x_flag, args=(high_fodmap_keywords,))

**Valores nutricionales**

In [16]:
#Valores nutricionales "sanos" y restricciones alimentarias, pueden ser adaptables según el usuario
#https://www-verywellfit-com.translate.goog/what-does-based-on-a-2000-calorie-diet-mean-4099137?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=sge#:~:text=Based%20on%20the%20DV%20information,or%201200%20calories%20from%20carbohydrates

Daily_values = {"Calories": 2000,
                "FatContent": 78,
                "SaturatedFatContent": 20,
                "CholesterolContent": 300,
                "SodiumContent": 2300,
                "CarbohydrateContent": 275,
                "FiberContent": 28,
                "SugarContent": 40,
                "ProteinContent": 50}

Meal_values = Daily_values.copy()
for value in Meal_values:
    Meal_values[value] = Meal_values[value]/3 #Son 3 comidas al día

In [17]:
recipes_healthy = recipes.copy()

for key, value in Meal_values.items():
    recipes_healthy[key] = recipes_healthy[key]/recipes_healthy["RecipeServings"]
    recipes_healthy = recipes_healthy[recipes_healthy[key] <= value]

recipes_healthy.drop("RecipeServings", axis=1, inplace=True)

In [18]:
recipes_healthy.reset_index(drop=True, inplace=True)
recipes_healthy.head()

,RecipeId,Name,AuthorId,RecipeCategory,Keywords,RecipeIngredientParts,AggregatedRating,Calories,FatContent,SaturatedFatContent,...,is_lactose_free,is_vegan,is_gluten_free,is_nut_free,is_soy_free,is_shellfish_free,is_halal,is_kosher,is_alcohol_free,is_high_fodmap
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,42.725000,0.6250,0.325000,...,0,0,1,1,1,1,1,1,1,0
1,39,Biryani,1567,Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,185.116667,9.8000,2.766667,...,0,0,1,0,1,1,1,1,1,0
2,42,Cabbage Soup,1538,Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,25.900000,0.1000,0.025000,...,1,1,1,1,1,1,1,1,1,0
3,43,Best Blackbottom Pie,34879,Pie,"c(""Dessert"", ""Weeknight"", ""Stove Top"", ""< 4 Ho...","c(""graham cracker crumbs"", ""sugar"", ""butter"", ...",1.0,54.737500,2.4125,1.362500,...,0,0,1,1,1,1,0,0,0,0
4,45,Buttermilk Pie With Gingersnap Crumb Crust,1580,Pie,"c(""Dessert"", ""Healthy"", ""Weeknight"", ""Oven"", ""...","c(""sugar"", ""margarine"", ""egg"", ""flour"", ""salt""...",4.0,28.500000,0.8875,0.212500,...,0,0,1,1,1,1,0,1,0,0


**KNN**

Recipes

In [19]:
def preprocess_list_column(col):
    return col.str.replace(r'[\[\]"]', '', regex=True).str.replace(',', ' ').str.lower()

recipes_healthy['text'] = preprocess_list_column(recipes_healthy['RecipeIngredientParts'].astype(str)) + " " + \
                 preprocess_list_column(recipes_healthy['Keywords'].astype(str))

In [20]:
numeric_features = [
    "Calories", "FatContent", "SaturatedFatContent", "CarbohydrateContent",
    "CholesterolContent", "SodiumContent", "ProteinContent", "FiberContent",
    'SugarContent', "AggregatedRating", 'is_vegetarian', 'is_lactose_free',
    'is_vegan', 'is_gluten_free', 'is_nut_free', 'is_soy_free', 'is_shellfish_free', 'is_halal',
    'is_kosher', 'is_alcohol_free', 'is_high_fodmap'
]

tfidf = TfidfVectorizer(max_features=300, stop_words="english")
scaler = StandardScaler()

In [21]:
preprocessor = ColumnTransformer(transformers=[
    ('tfidf', tfidf, 'text'),
    ('num', scaler, numeric_features)
])

X = preprocessor.fit_transform(recipes_healthy)

In [22]:
model_knn = NearestNeighbors(metric='cosine', algorithm='auto')
model_knn.fit(X)

NearestNeighbors(metric='cosine')

Reviews

In [23]:
og_reviews = pd.read_csv('reviews.csv')
og_reviews.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [24]:
reviews = og_reviews[["RecipeId", "AuthorId", "Rating"]].copy()

In [25]:
reviews_good = reviews[reviews["Rating"] >= 5]
reviews_count = reviews_good['AuthorId'].value_counts()
authors_to_keep = reviews_count[reviews_count >= 5].index
reviews = reviews[reviews['AuthorId'].isin(authors_to_keep)].copy()
reviews.reset_index(drop=True, inplace=True)

In [26]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

reviews['user_idx'] = user_encoder.fit_transform(reviews['AuthorId'])
reviews['item_idx'] = item_encoder.fit_transform(reviews['RecipeId'])

sparse_ratings = csr_matrix(
    (reviews['Rating'], (reviews['user_idx'], reviews['item_idx']))
)

In [27]:
model_knn_ratings = NearestNeighbors(metric='cosine', algorithm='auto')
model_knn_ratings.fit(sparse_ratings)

NearestNeighbors(metric='cosine')

In [28]:
def predecir_rating_usuario_sparse(user_id, recipe_id, k=5):
    if user_id not in user_encoder.classes_ or recipe_id not in item_encoder.classes_:
        return np.nan

    user_idx = user_encoder.transform([user_id])[0]
    recipe_idx = item_encoder.transform([recipe_id])[0]

    distances, indices = model_knn_ratings.kneighbors(sparse_ratings[user_idx], n_neighbors=k+1)

    sum_rating = 0
    sum_weights = 0

    for dist, idx in zip(distances[0][1:], indices[0][1:]):
        neighbor_rating = sparse_ratings[idx, recipe_idx]
        if neighbor_rating > 0:
            weight = 1 - dist
            sum_rating += neighbor_rating * weight
            sum_weights += weight

    return sum_rating / sum_weights if sum_weights > 0 else 0

In [29]:
def recomendar_hibrido_puntaje_sparse(user_id, receta_base_id, recipes_df, n=10, alpha=0.5):
    if receta_base_id not in recipes_df['RecipeId'].values:
        return None

    idx_base = recipes_df[recipes_df['RecipeId'] == receta_base_id].index[0]
    distances, indices = model_knn.kneighbors(X[idx_base], n_neighbors=15)

    resultados = []
    for idx, dist in zip(indices[0][1:], distances[0][1:]):
        rec = recipes_df.iloc[idx]
        similarity = 1 - dist
        pred_rating = predecir_rating_usuario_sparse(user_id, rec['RecipeId'])
        if not np.isnan(pred_rating):
            score = alpha * similarity + (1 - alpha) * (pred_rating / 5.0)
            resultados.append((rec['RecipeId'], rec['Name'], similarity, pred_rating, score))

    resultados.sort(key=lambda x: -x[4])
    top = resultados[:n]

    return pd.DataFrame(top, columns=["RecipeId", "Name", "Similarity", "PredictedRating", "HybridScore"])

In [30]:
def get_all_recs(reviews, recipes):
    recommendations_list = []
    total_recommendations = []

    for author_id in reviews['AuthorId'].unique():
        user_reviews = reviews[reviews['AuthorId'] == author_id]
        user_reviews = user_reviews[user_reviews["Rating"] >= 5].head()
        if len(user_reviews) == 0:
            continue

        for recipe_id in user_reviews['RecipeId'].unique():
            recommendations = recomendar_hibrido_puntaje_sparse(user_id=author_id,
                                                                receta_base_id=recipe_id,
                                                                recipes_df=recipes, n=5)
            if recommendations is not None:
                recommendations['AuthorId'] = author_id
                recommendations_list.append(recommendations)

        if recommendations_list:
            all_recommendations = pd.concat(recommendations_list, ignore_index=True)
            top_recommendations = all_recommendations.sort_values(by=['HybridScore'], ascending=[False])
            top_5_per_user = top_recommendations.head()
            total_recommendations.append(top_5_per_user)
            recommendations_list = []

    total_recommendations_df = pd.concat(total_recommendations, ignore_index=True)
    return total_recommendations_df

In [32]:
num_authors = int(len(reviews['AuthorId'].unique()) * 0.1)
sampled_authors = np.random.choice(reviews['AuthorId'].unique(), size=num_authors, replace=False)
sampled_reviews = reviews[reviews['AuthorId'].isin(sampled_authors)].copy()

In [33]:
recs = get_all_recs(sampled_reviews, recipes_healthy)

In [34]:
recs

,RecipeId,Name,Similarity,PredictedRating,HybridScore,AuthorId
0,22030,Angel Hair Pasta with Chicken and Veggies,0.981333,0.0,0.490666,1634
1,216975,Angel Hair Pasta With Chicken,0.963026,0.0,0.481513,1634
2,66318,Creamy Chicken &amp; Spinach,0.956978,0.0,0.478489,1634
3,116796,Garlic Chicken Pasta Primavera,0.954323,0.0,0.477162,1634
4,80971,Tuna Pasta with Artichoke and Semi-Sun-Dried T...,0.942461,0.0,0.471231,1634
...,...,...,...,...,...,...
10489,182387,Mom's Pecan Pie,0.968348,0.0,0.484174,2002409491
10490,2583,The Camellia Grill's Pecan Pie,0.967682,0.0,0.483841,2002409491
10491,96937,Pecan Streusel Cake,0.964824,0.0,0.482412,2002409491
10492,398239,Mimi's Pecan Pies,0.964388,0.0,0.482194,2002409491


In [35]:
recs.to_csv('recommendations.csv', index=False)